## **_Zajem gibanja telesa iz videa - uvoz v program Blender_** 

[Blender](https://www.blender.org/) je brezplačen, odprtokoden program za ustvarjanje 3D modelov, animiacij in grafik.

Z različico 3.3, so avtorji programa Blender uporabnikom omogočili pisanje lastnih skript v programskem jeziku kode, kar bomo uporabili za uvoz prečiščenih podatkov zajema gibanja.

Za uvažanje zajetega gibanja, je bilo potrebno podatke ločiti na posamezne mejnike. To nam je olajšalo avtomatizacijo procesa ustvarjanja telesa, ki predstavlja ta mejnik, in uvažanja podatkov o lokaciji le-tega, ter ustvarjanja animacije. Naša skripta uvozi podatke vseh mejnikov, shranjenih v datoteki _landmark\_data_, ustvari krogle in pravilno nastavi lokacijo krogel, ki predstavljajo zaznane mejnike.

Krogle lahko uporabimo kot "sklepe", ki vodijo animacijo 3D telesa. Celoten video v višji kvaliteti je dostopen na GitHub repozitoriju v mapi 'render'. Blender projekt je prav tako dostopen na GitHub repozitoriju, v 

![Animacija](./render/MoCapRenderClip.gif)

In [ ]:
#Python skripta, uporabljena v programu Blender, ki ustvari krogle in jim priredi animacijo

import bpy
import csv
import os

def parse_csv_data(path):
    """
        Parses individual landmark csv, returns as list of lists for each frame of the animation
    """
    with open(path, 'r') as f:
        reader = csv.reader(f)
        mocap_data = []
        for r in reader:
            frame = int(r[0])
            x = float(r[1])
            y = float(r[2])
            z = float(r[3])

            mocap_data.append((frame, x, y, z))
    
    return mocap_data

def create_sphere(x, y, z):
    bpy.ops.object.select_all(action='DESELECT')    #Deselects any previously selected objects in our scene
    bpy.ops.mesh.primitive_uv_sphere_add(radius=0.3, location=(x,y,z))    #Creates primitive UV Sphere at the provided location
    sphere = bpy.context.object
    return sphere

def create_animation(sphere, mocap_data):
    sphere.animation_data_create()    #Creates fresh animation queue for our object
    sphere.animation_data.action = bpy.data.actions.new(name="MocapAnimation")
    fcurves = [sphere.animation_data.action.fcurves.new(data_path='location', index=i) for i in range(3)] #Fcurves represent the path each object takes
    
    for frame, x, y, z in mocap_data:
        for i, fcurve in enumerate(fcurves):
            keyframe = frame + 1
            value = (x, y, z)[i]
            fcurve.keyframe_points.insert(keyframe, value)    #Keyframes represent our positional information for each frame in our animation
            

csv_path = r'C:/Users/miham/Documents/Faks/VGRAJENI SISTEMI/landmark_data'
csv_names = os.listdir(csv_path)

for i in range(1, len(csv_names)):
    #iterates through each of our landmark files
    path = csv_path + "/" + csv_names[i]
    
    mocap_data = parse_csv_data(path)
    sphere = create_sphere(mocap_data[0][1], mocap_data[0][2], mocap_data[0][3])
    
    create_animation(sphere, mocap_data)



Po zagonu skripte, bomo na sceni imeli 17 krogel, ki predstavljajo posamezne zaznane mejnike. Vsaka izmed krogel bo imela tudi animacijo, ki predstavlja zajeto gibanje mejnika.

![blender](./images/blender-scene1.png)

Krogle lahko nato uporabimo kot "sklepe", ki vodijo animacijo modela telesa, ki ga ustvarimo sami. Ustvarjeno telo je preprosto - vsako vozlišče predstavlja en sklep.

![Blender2](./images/blender-scene2.png)

## **_Viri_**
 - Blender API Dokumentacija, _https://docs.blender.org/api/current/_